In [2]:
import pandas as pd
from datetime import datetime, timedelta

# --- Part 1: Price Estimation Model (from previous task) ---
# This section loads the historical data and prepares the price estimation function.

try:
    # Load the dataset
    df = pd.read_csv('Nat_Gas.csv')
    df['Dates'] = pd.to_datetime(df['Dates'])

    # Prepare data for a simple interpolation/extrapolation model
    # For a prototype, a linear interpolation is sufficient and robust.
    df = df.sort_values(by='Dates')
    df.set_index('Dates', inplace=True)

except FileNotFoundError:
    print("Error: Nat_Gas.csv not found. Please ensure the file is in the correct directory.")
    # Create a dummy dataframe to allow the script to run, but estimations will fail.
    df = pd.DataFrame(columns=['Prices'])


def estimate_price(target_date_str):
    """
    Estimates the price of natural gas for a given date using linear interpolation
    and nearest-neighbor extrapolation.
    :param target_date_str: Date in 'YYYY-MM-DD' format.
    :return: Estimated price as a float, or None on error.
    """
    try:
        target_date = pd.to_datetime(target_date_str)
        
        # Reindex the dataframe to include the target date and then interpolate
        all_dates = df.index.union([target_date])
        reindexed_df = df.reindex(all_dates)
        interpolated_df = reindexed_df.interpolate(method='time')
        
        # If the date is outside the historical range, fill with the nearest value
        interpolated_df = interpolated_df.bfill().ffill()
        
        return interpolated_df.loc[target_date]['Prices']

    except Exception as e:
        print(f"Error estimating price for {target_date_str}: {e}")
        return None

# --- Part 2: Gas Storage Contract Pricing Function ---

def price_gas_storage_contract(
    injection_dates,
    injection_volumes,
    withdrawal_dates,
    withdrawal_volumes,
    max_volume,
    storage_cost_rate_per_month
):
    """
    Calculates the value of a gas storage contract based on cash flows.

    Parameters:
    - injection_dates (list): List of dates ('YYYY-MM-DD') for gas injection.
    - injection_volumes (list): List of gas volumes to inject.
    - withdrawal_dates (list): List of dates ('YYYY-MM-DD') for gas withdrawal.
    - withdrawal_volumes (list): List of gas volumes to withdraw.
    - max_volume (float): Maximum storage capacity.
    - storage_cost_rate_per_month (float): Cost per unit of volume per month.

    Returns:
    - float: The total net value of the contract.
    - str: An error message if the contract is invalid.
    """
    # Create a list of all transactions (positive for injection, negative for withdrawal)
    injections = [{'date': pd.to_datetime(d), 'volume': v} for d, v in zip(injection_dates, injection_volumes)]
    withdrawals = [{'date': pd.to_datetime(d), 'volume': -v} for d, v in zip(withdrawal_dates, withdrawal_volumes)]
    
    transactions = sorted(injections + withdrawals, key=lambda x: x['date'])

    total_purchase_cost = 0.0
    total_sales_revenue = 0.0
    total_storage_cost = 0.0
    current_volume = 0.0
    
    if not transactions:
        return 0.0, "No transactions provided."

    last_transaction_date = transactions[0]['date']

    # Daily storage cost rate
    daily_storage_rate = storage_cost_rate_per_month / 30.44  # Average days in a month

    # --- Process transactions and calculate costs/revenues ---
    for trans in transactions:
        # 1. Calculate storage cost for the period since the last transaction
        days_in_period = (trans['date'] - last_transaction_date).days
        storage_cost_for_period = current_volume * days_in_period * daily_storage_rate
        total_storage_cost += storage_cost_for_period
        
        # 2. Estimate the price for the transaction date
        price = estimate_price(trans['date'].strftime('%Y-%m-%d'))
        if price is None:
            return None, f"Could not estimate price for date {trans['date'].date()}"

        # 3. Process the transaction (injection or withdrawal)
        if trans['volume'] > 0:  # Injection
            # Check if there's enough space
            if current_volume + trans['volume'] > max_volume:
                return None, f"Contract invalid: Injection on {trans['date'].date()} exceeds max volume of {max_volume}."
            
            total_purchase_cost += trans['volume'] * price
            current_volume += trans['volume']
        
        else:  # Withdrawal
            volume_to_withdraw = abs(trans['volume'])
            # Check if there's enough gas to withdraw
            if current_volume < volume_to_withdraw:
                return None, f"Contract invalid: Withdrawal on {trans['date'].date()} exceeds available volume of {current_volume}."

            total_sales_revenue += volume_to_withdraw * price
            current_volume -= volume_to_withdraw
            
        last_transaction_date = trans['date']

    # --- Calculate final contract value ---
    net_value = total_sales_revenue - total_purchase_cost - total_storage_cost

    print("--- Contract Valuation Summary ---")
    print(f"Total Sales Revenue: ${total_sales_revenue:,.2f}")
    print(f"Total Purchase Cost: ${total_purchase_cost:,.2f}")
    print(f"Total Storage Cost:  ${total_storage_cost:,.2f}")
    print("----------------------------------")
    print(f"Net Contract Value:  ${net_value:,.2f}")
    print("----------------------------------")
    
    return net_value, None


# --- Part 3: Test Case Example ---
if __name__ == "__main__":
    print("Running Gas Storage Contract Pricing Model...\n")

    # --- Sample Contract Inputs ---
    sample_injection_dates = ['2023-06-01', '2023-07-01']
    sample_injection_volumes = [10000, 10000]  # in MMBtu

    sample_withdrawal_dates = ['2023-12-15', '2024-01-20']
    sample_withdrawal_volumes = [8000, 12000] # in MMBtu

    sample_max_volume = 25000  # Max storage capacity in MMBtu
    sample_storage_cost_rate = 0.10  # $0.10 per MMBtu per month

    # --- Price the contract ---
    contract_value, error = price_gas_storage_contract(
        injection_dates=sample_injection_dates,
        injection_volumes=sample_injection_volumes,
        withdrawal_dates=sample_withdrawal_dates,
        withdrawal_volumes=sample_withdrawal_volumes,
        max_volume=sample_max_volume,
        storage_cost_rate_per_month=sample_storage_cost_rate
    )

    if error:
        print(f"\nError pricing contract: {error}")
    else:
        print(f"\nThe final estimated value of the contract is ${contract_value:,.2f}")


Running Gas Storage Contract Pricing Model...

--- Contract Valuation Summary ---
Total Sales Revenue: $251,974.19
Total Purchase Cost: $221,061.29
Total Storage Cost:  $13,377.14
----------------------------------
Net Contract Value:  $17,535.77
----------------------------------

The final estimated value of the contract is $17,535.77


C:\Users\HP\AppData\Local\Temp\ipykernel_10212\3115355458.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])
